In [1]:
import sys
import sqlite3
from PyQt5.QtWidgets import QApplication, QMainWindow
from PyQt5.QtGui import QStandardItemModel
from PyQt5.uic import loadUi
from PyQt5.QtCore import pyqtSlot, Qt
from dbhandler import DBHandler
from cetizen import PnoCrawler, ReleasePriceCrawler, UsedPriceCrawler
from datetime import datetime

__author__ = 'Sangjin Lee <lee3jjang@gmail.com>'

In [2]:
class CetizenWindow(QMainWindow):

    def __init__(self, parent=None):
        super().__init__(parent)
        conn = sqlite3.connect('cetizen.db')
        self.db = DBHandler(conn)
        self.ui = loadUi('cetizen.ui', self)

        self.logger.setText('')
        self.table.setAlternatingRowColors(True)
        #self.table.setRootIsDecorated(False)

        self.setWindowTitle('Cetizen Crawler')
        self.ui.show()

    @pyqtSlot()
    def slot_pno_select(self):
        df = self.db.getTableAll('상품정보')
        n = df.shape[0]
        rows = QStandardItemModel(n, 4, self)
        rows.setHeaderData(0, Qt.Horizontal, '통신사')
        rows.setHeaderData(1, Qt.Horizontal, '상품코드')
        rows.setHeaderData(2, Qt.Horizontal, '상품명')
        rows.setHeaderData(3, Qt.Horizontal, '모델명')
        for i in range(n):
            for j in range(4):
                rows.setData(rows.index(i, j), df.iloc[i, j])
        self.table.setModel(rows)

    @pyqtSlot()
    def slot_pno_insert(self):
        self.logger.append('-- 상품정보    수집 시작 -- {}'.format(datetime.now()))
        pno_crawler = PnoCrawler()
        df = pno_crawler.crawling(save=True)
        self.logger.append('-- 상품정보    수집 완료 -- {}'.format(datetime.now()))
        self.db.insTable('상품정보', df)
        self.logger.append('-- 상품정보    적재 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_pno_delete(self):
        self.db.delTableAll('상품정보')
        self.logger.append('-- 상품정보    삭제 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_rel_select(self):
        df = self.db.getTableAll('출고가정보')
        n = df.shape[0]
        rows = QStandardItemModel(n, 3, self)
        rows.setHeaderData(0, Qt.Horizontal, '상품코드')
        rows.setHeaderData(1, Qt.Horizontal, '기준일자')
        rows.setHeaderData(2, Qt.Horizontal, '출고가')
        for i in range(n):
            for j in range(3):
                rows.setData(rows.index(i, j), str(df.iloc[i, j]))
        self.table.setModel(rows)

    @pyqtSlot()
    def slot_rel_insert(self):
        self.logger.append('-- 출고가정보 수집 시작 -- {}'.format(datetime.now()))
        pno = list(self.db.getTableAll('상품정보')['PNO'])
        release_price_crawler = ReleasePriceCrawler(pno)
        df = release_price_crawler.crawling(save=True)
        self.logger.append('-- 출고가정보 수집 완료 -- {}'.format(datetime.now()))
        self.db.insTable('출고가정보', df)
        self.logger.append('-- 출고가정보 적재 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_rel_delete(self):
        self.db.delTableAll('출고가정보')
        self.logger.append('-- 출고가정보 삭제 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_used_select(self):
        df = self.db.getTableAll('중고가정보')
        n = df.shape[0]
        rows = QStandardItemModel(n, 5, self)
        rows.setHeaderData(0, Qt.Horizontal, '상품코드')
        rows.setHeaderData(1, Qt.Horizontal, '기준일자')
        rows.setHeaderData(2, Qt.Horizontal, '저가')
        rows.setHeaderData(3, Qt.Horizontal, '중간')
        rows.setHeaderData(4, Qt.Horizontal, '고가')
        for i in range(n):
            for j in range(5):
                rows.setData(rows.index(i, j), str(df.iloc[i, j]))
        self.table.setModel(rows)

    @pyqtSlot()
    def slot_used_insert(self):
        self.logger.append('-- 중고가정보 수집 시작 -- {}'.format(datetime.now()))
        pno = list(self.db.getTableAll('상품정보')['PNO'])
        used_price_crawler = UsedPriceCrawler(pno)
        df = used_price_crawler.crawling(save=True)
        self.logger.append('-- 중고가정보 수집 완료 -- {}'.format(datetime.now()))
        self.db.insTable('중고가정보', df)
        self.logger.append('-- 중고가정보 적재 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_used_delete(self):
        self.db.delTableAll('중고가정보')
        self.logger.append('-- 중고가정보 삭제 완료 -- {}'.format(datetime.now()))

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    cetizen = CetizenWindow()
    sys.exit(app.exec_())

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from dbhandler import DBHandler
from datetime import datetime

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
conn = sqlite3.connect('cetizen.db')
db = DBHandler(conn)

#### 1. 가정설정

In [4]:
가정_전체 = db.getTableAll('가정')
계약정보_전체 = db.getTableAll('계약정보')

가정ID = 'TEST_ASSUM_001'
계약정보ID = 'TEST_CONT_001'

가정 = 가정_전체.query('가정ID == "{}"'.format(가정ID))
계약정보 = 계약정보_전체.query('계약정보ID == "{}"'.format(계약정보ID))

# 계약정보 입력
월납보험료 = 계약정보['월납보험료'].values[0]
출고가 = 계약정보['출고가'].values[0]
반납시점 = 계약정보['반납시점'].values[0]
보장율 = 계약정보['보장율'].values[0]/100
손실한도 = 계약정보['손실한도'].values[0]/100
공임비 = 계약정보['공임비'].values[0]
공동인수율 = 계약정보['공동인수율'].values[0]/100
대리점수수료율 = 계약정보['대리점수수료율'].values[0]/100
출재율 = 계약정보['출재율'].values[0]/100
출재수수료율 = 계약정보['출재수수료율'].values[0]/100

# 가정 입력
총가입자수 = 가정['총가입자수'].values[0]
월해지율 = 가정['월해지율'].values[0]/100
매각가율 = 가정['매각가율'].values[0]/100
유지자기변율 = 가정['유지자기변율'].values[0]/100
기타사업비율 = 가정['기타사업비율'].values[0]/100
잔존율 = 가정['잔존율'].values[0]/100

#### 2. 가치평가

In [5]:
# 해지율
해지율 = np.ones(반납시점)*월해지율

# 유지율
유지율 = np.ones(반납시점+1)
for i in range(1, 반납시점+1):
    유지율[i] = 유지율[i-1]-해지율[i-1]

# 기변율
기변율 = np.zeros(반납시점)
기변율[-1] = 유지율[-1]*유지자기변율
기변율

# 손실율
손실율 = 보장율 - 잔존율*매각가율

In [6]:
# 기말유지자
기말유지자 = 총가입자수*유지율

# 기변자
기변자 = np.zeros_like(기말유지자)
기변자[-1] = 기말유지자[-1]*유지자기변율

# 원수보험료, 출재보험료, 보유보험료
원수보험료 = 월납보험료*기말유지자
원수보험료[-1] = 0
출재보험료 = 원수보험료*출재율
보유보험료 = 원수보험료-출재보험료

# 원수보험금, 출재보험금, 보유보험료
원수보험금 = np.zeros_like(원수보험료)
원수보험금[-1] = max(min((출고가*손실율+공임비)*기변자[-1], 원수보험료.sum()*손실한도), 0)
출재보험금 = 원수보험금*출재율
보유보험금 = 원수보험금-출재보험금

# 대리점수수료, 출재수수료, 기타사업비, 원수사업비, 보유사업비
대리점수수료 = 원수보험료*대리점수수료율
출재수수료 = 출재보험료*출재수수료율
기타사업비 = 원수보험료*기타사업비율
원수사업비 = 대리점수수료+기타사업비
보유사업비 = 원수사업비-출재수수료

# 현금유입, 현금유출, 순현금흐름
현금유입 = 원수보험료+출재보험금+출재수수료
현금유출 = 원수보험금+출재보험료+출재수수료+기타사업비
순현금흐름 = 현금유입-현금유출

In [7]:
가치평가 = pd.DataFrame({
    '시점':np.arange(반납시점+1),
    '기말유지자':기말유지자,
    '기변자':기변자,
    '원수보험료':원수보험료,
    '출재보험료':출재보험료,
    '보유보험료':보유보험료,
    '원수보험금':원수보험금,
    '출재보험금':출재보험금,
    '보유보험금':보유보험금,
    '대리점수수료':대리점수수료,
    '출재수수료':출재수수료,
    '기타사업비':기타사업비,
    '원수사업비':원수사업비,
    '보유사업비':보유사업비,
    '현금유입':현금유입,
    '현금유출':현금유출,
    '순현금흐름':순현금흐름})

가치평가['가정ID'] = 가정ID
가치평가['계약정보ID'] = 계약정보ID
가치평가['평가ID'] = 'TEST_VAL_001'
가치평가['LAST_MODIFIED_BY'] = 'RVI_PHONE'
가치평가['LAST_UPDATE_DATE'] = datetime.now().strftime('%Y-%m-%d')

가치평가_열 = db.getColumn('가치평가')
가치평가 = 가치평가[가치평가_열]
가치평가

,평가ID,계약정보ID,가정ID,시점,기말유지자,기변자,원수보험료,출재보험료,보유보험료,원수보험금,...,대리점수수료,출재수수료,기타사업비,원수사업비,보유사업비,현금유입,현금유출,순현금흐름,LAST_MODIFIED_BY,LAST_UPDATE_DATE
0,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,0,"4,500.00",0.00,"21,600,000.00","7,560,000.00","14,040,000.00",0.00,...,"1,080,000.00","1,512,000.00","1,080,000.00","2,160,000.00","648,000.00","23,112,000.00","10,152,000.00","12,960,000.00",RVI_PHONE,2019-02-06
1,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,1,"4,432.50",0.00,"21,276,000.00","7,446,600.00","13,829,400.00",0.00,...,"1,063,800.00","1,489,320.00","1,063,800.00","2,127,600.00","638,280.00","22,765,320.00","9,999,720.00","12,765,600.00",RVI_PHONE,2019-02-06
2,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,2,"4,365.00",0.00,"20,952,000.00","7,333,200.00","13,618,800.00",0.00,...,"1,047,600.00","1,466,640.00","1,047,600.00","2,095,200.00","628,560.00","22,418,640.00","9,847,440.00","12,571,200.00",RVI_PHONE,2019-02-06
3,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,3,"4,297.50",0.00,"20,628,000.00","7,219,800.00","13,408,200.00",0.00,...,"1,031,400.00","1,443,960.00","1,031,400.00","2,062,800.00","618,840.00","22,071,960.00","9,695,160.00","12,376,800.00",RVI_PHONE,2019-02-06
4,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,4,"4,230.00",0.00,"20,304,000.00","7,106,400.00","13,197,600.00",0.00,...,"1,015,200.00","1,421,280.00","1,015,200.00","2,030,400.00","609,120.00","21,725,280.00","9,542,880.00","12,182,400.00",RVI_PHONE,2019-02-06
5,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,5,"4,162.50",0.00,"19,980,000.00","6,993,000.00","12,987,000.00",0.00,...,"999,000.00","1,398,600.00","999,000.00","1,998,000.00","599,400.00","21,378,600.00","9,390,600.00","11,988,000.00",RVI_PHONE,2019-02-06
6,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,6,"4,095.00",0.00,"19,656,000.00","6,879,600.00","12,776,400.00",0.00,...,"982,800.00","1,375,920.00","982,800.00","1,965,600.00","589,680.00","21,031,920.00","9,238,320.00","11,793,600.00",RVI_PHONE,2019-02-06
7,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,7,"4,027.50",0.00,"19,332,000.00","6,766,200.00","12,565,800.00",0.00,...,"966,600.00","1,353,240.00","966,600.00","1,933,200.00","579,960.00","20,685,240.00","9,086,040.00","11,599,200.00",RVI_PHONE,2019-02-06
8,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,8,"3,960.00",0.00,"19,008,000.00","6,652,800.00","12,355,200.00",0.00,...,"950,400.00","1,330,560.00","950,400.00","1,900,800.00","570,240.00","20,338,560.00","8,933,760.00","11,404,800.00",RVI_PHONE,2019-02-06
9,TEST_VAL_001,TEST_CONT_001,TEST_ASSUM_001,9,"3,892.50",0.00,"18,684,000.00","6,539,400.00","12,144,600.00",0.00,...,"934,200.00","1,307,880.00","934,200.00","1,868,400.00","560,520.00","19,991,880.00","8,781,480.00","11,210,400.00",RVI_PHONE,2019-02-06


In [8]:
db.insTable('가치평가', 가치평가)